In [63]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io

In [64]:
dataset = io.loadmat('datasets/emnist-letters.mat')
# matlab_structs / numpy structured arrays. yay
# they have a dtype (structure) and every element adheres to it :)

In [65]:
struct_dataset = dataset['dataset'][0,0]

train_images, train_labels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
test_images, test_labels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(124800, 784) (20800, 784) (124800, 1) (20800, 1)


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels))

# Best = 94.32% validation acc

Epoch 1/15
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6708 - loss: 1.0788